# 05. CUB-200 Experiments

CUB-200 experiments with ResNet-50 transfer learning.

**Settings:** 3 clients, 75 rounds, batch size 16

**Prerequisite:** Download CUB-200 (see RUNPOD_SETUP_GUIDE.md)

In [ ]:
import sys, os, time, json
from datetime import datetime
if os.path.basename(os.getcwd()) == 'notebooks': os.chdir('..')
sys.path.insert(0, os.getcwd())

import torch
import numpy as np
from experiments.run_experiments import ExperimentConfig, ExperimentRunner

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Check CUB-200
if not os.path.exists("./data/CUB_200_2011"):
    raise RuntimeError("CUB-200 not found. See RUNPOD_SETUP_GUIDE.md")
print("✓ CUB-200 found")

In [ ]:
# Configuration
CONFIG = {"num_clients": 3, "num_rounds": 75, "batch_size": 16, "learning_rate": 0.001}
SEEDS = [42, 123, 456]
DISTRIBUTIONS = [("iid", 1.0), ("noniid", 0.5), ("noniid", 0.1)]

ATTACKS = {
    "none": {"enabled": False},
    "label_flip": {"enabled": True, "type": "label_flip", "malicious_clients": [0], "poison_ratio": 0.3},
    "backdoor": {"enabled": True, "type": "backdoor", "malicious_clients": [0], "poison_ratio": 0.1},
}

DEFENSES = {
    "none": {"enabled": False},
    "krum": {"enabled": True, "type": "krum", "num_malicious": 1},
    "fltrust": {"enabled": True, "type": "fltrust"},
}

In [ ]:
# Run experiments
RESULTS_DIR = f"./experiments/cub200_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(RESULTS_DIR, exist_ok=True)
runner = ExperimentRunner(RESULTS_DIR)

experiments = [("none", "none", d, a) for d, a in DISTRIBUTIONS]
for atk in ["label_flip", "backdoor"]:
    for dfn in DEFENSES:
        for d, a in DISTRIBUTIONS:
            experiments.append((atk, dfn, d, a))

all_results = []
for i, (atk, dfn, dist, alpha) in enumerate(experiments):
    seed_results = []
    for seed in SEEDS:
        print(f"[{i+1}/{len(experiments)}] {atk}+{dfn} | {dist}(α={alpha}) | seed={seed}")
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        
        cfg = ExperimentConfig(
            name=f"cub200_{atk}_{dfn}_{dist}_s{seed}", dataset="cub200",
            num_clients=CONFIG["num_clients"], num_rounds=CONFIG["num_rounds"],
            local_epochs=1, batch_size=CONFIG["batch_size"], learning_rate=CONFIG["learning_rate"],
            partition=dist, seed=seed,
            attack_enabled=ATTACKS[atk].get("enabled", False), attack_type=ATTACKS[atk].get("type", "none"),
            malicious_clients=ATTACKS[atk].get("malicious_clients", []),
            defense_enabled=DEFENSES[dfn].get("enabled", False), defense_type=DEFENSES[dfn].get("type", "none"),
        )
        try:
            result = runner.run_simulation(cfg)
            seed_results.append({"seed": seed, "accuracy": result.final_accuracy})
        except Exception as e:
            print(f"  ERROR: {e}")
    
    if seed_results:
        accs = [r["accuracy"] for r in seed_results]
        all_results.append({"attack": atk, "defense": dfn, "dist": dist, "alpha": alpha,
                           "mean_acc": np.mean(accs), "std_acc": np.std(accs, ddof=1) if len(accs)>1 else 0})
        print(f"  → {np.mean(accs)*100:.2f}% ± {np.std(accs, ddof=1)*100:.2f}%")

In [ ]:
# Save results
with open(os.path.join(RESULTS_DIR, "cub200_results.json"), 'w') as f:
    json.dump(all_results, f, indent=2)
print(f"Results saved to {RESULTS_DIR}")